In [ ]:
%matplotlib notebook
from importlib import reload
import tables
import numpy as np
import dashi
dashi.visual()
import yaml

import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from CHECLabPy.plotting.camera import CameraImage, CameraImageImshow
from CHECLabPy.utils.mapping import get_clp_mapping_from_tc_mapping, \
    get_superpixel_mapping, get_tm_mapping
import ssdaq
from ssm.core.pchain import ProcessingChain, ProcessingModule

In [ ]:
!python ssm/scripts/slowsignal_simulation.py -h

In [ ]:
!python ssm/scripts/slowsignal_simulation.py

## Load the simulation data

In [ ]:
from ssm.core.sim_io import DataReader
reader = DataReader('VegaTrack40minFlatNSB.hdf')
print(reader)

print('Simulation config:',reader.sim_attr_dict)


In [ ]:
res = []
res_r = []
s_pos = []
times = []
for r in reader.read():
    res.append(r.flatten())
    res_r.append(reader.raw_data.flatten())
    times.append(reader.cpu_t)
    s_pos.append(reader.source_pos)
s_pos = np.array(s_pos)

# Look at the data

## Plot of average amplitude over all data
This effectively makes 'long exposure' plot

In [ ]:
###### from CHECLabPy.core.io import DL1Reader, TIOReader
from CHECLabPy.plotting.camera import CameraImage, CameraImageImshow
from CHECLabPy.utils.mapping import get_clp_mapping_from_tc_mapping, \
    get_superpixel_mapping, get_tm_mapping
from target_calib import CameraConfiguration
c = CameraConfiguration("1.1.0")
m = c.GetMapping()
xpix = np.array(m.GetXPixVector())
ypix = np.array(m.GetYPixVector())
size = m.GetSize()
camera = CameraImage(xpix, ypix, size)
camera.add_colorbar('$\log_{10}$(amplitude/mV)')
im=(np.sum(res,axis=0)/len(res)) 
im = np.log10(im+0.0001)
camera.image =im
plt.plot(s_pos[0,:,0]*1e-3,s_pos[0,:,1]*1e-3,'*')


## First frame
Here we plot the first frame and plot on top hotspots (>170mV) and star positions

In [ ]:
camera = CameraImage(xpix, ypix, size)
camera.add_colorbar('$\log_{10}$(amplitude/mV)')
from ssdaq.core.data_classes import ss_mappings
im=(np.sum(res,axis=0)/len(res)) 
im = res[0]
im = np.log10(im)
camera.image =im
peaks = np.where(res[0]>170)
plt.plot(xpix[peaks],ypix[peaks],'ro')
print("Number of peaks found {}".format(peaks[0].shape[0]))
plt.plot(s_pos[0,:,0]*1e-3,s_pos[0,:,1]*1e-3,'*')

## Make pixel neigbor map
Finding neigboring pixels

In [ ]:
from tqdm.auto import tqdm
neighbors = []#[[]]*2048
for i in range(2048):
    neighbors.append([])
for i in tqdm(range(2048),total=2048):
    for j in range(i+1,2048):

        dx = xpix[i]-xpix[j]
        dy = ypix[i]-ypix[j]
            
        if(np.sqrt(dx**2+dy**2)<11e-3):
            neighbors[i].append(j)
            neighbors[j].append(i)

## Build cluster and cluster evolutions
A first atempt at a clustering algorithm to reduce the number of pixels that will be used in the reconstruction

In [ ]:
def cluster_build(i,res,lot,visited):
    cluster = []
    visited.append(i)
    if(res[i]>lot):
        cluster.append(i)
        for n in neighbors[i]:
            if n in visited:
                continue
            cluster += cluster_build(n,res,lot,visited)
    return cluster
def find_clusters(res,upthreshold,lothreshold):
    peaks = np.where(res>upthreshold)
    clusters = []
    for peak in peaks[0]:
        in_c = False
        for c in clusters:
            if peak in c:
                in_c = True
                break
        
        if in_c:
            continue
        clusters.append(cluster_build(peak,res,lothreshold,[peak]))
    return clusters

def get_cluster_evolution(clusters,res,upthreshold,lothreshold):
    cmasks = []
    sress = []
    pix_is = []
    for p in tqdm(clusters,total=len(clusters)):
        peak = p[0]
        pix_ind = set(list(p))
        sres = []
        for r in res:
            if(r[peak]<90):
                break
            ci = cluster_build(peak,r,lothreshold,[peak])
            pix_ind = pix_ind.union(ci)

            cm = np.zeros(r.shape, dtype=bool)
            cm[ci] =True
            rr = r.copy()
            rr[~cm] = 0

            maxv = np.nanmax(rr)
            peak = np.nanargmax(rr)

            cmasks.append(ci)
            sres.append(rr)
        pix_is.append(np.array(list(pix_ind)))
        sress.append(sres)
    return sress,pix_is

clusters = find_clusters(res[0], 170,90)
sress,ipixs = get_cluster_evolution(clusters,res,170,90)
print("Found {} clusters".format(len(ipixs)))

## Plotting data from one cluster

In [ ]:
camera = CameraImage(xpix, ypix, size)
camera.add_colorbar('$\log_{10}$(amplitude/mV)')
im = np.zeros(res[0].shape)
for i in range(len(sress)):
    if(i==1):
        im+=(np.sum(sress[i],axis=0)/len(res)) 
im = np.log10(im+0.000001)
camera.image =im

plt.plot(s_pos[0,:,0]*1e-3,s_pos[0,:,1]*1e-3,'*')
